# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to yumupinglan@gmail.com and will expire on March 22, 2020.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1560257480.log


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.693225     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.737573     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [6]:
model_all.get("coefficients")

name,index,value,stderr
(intercept),None,274873.05595,None
bedrooms,None,0.0,None
bedrooms_square,None,0.0,None
bathrooms,None,8468.53108691,None
sqft_living,None,24.4207209824,None
sqft_living_sqrt,None,350.060553386,None
sqft_lot,None,0.0,None
sqft_lot_sqrt,None,0.0,None
floors,None,0.0,None
floors_square,None,0.0,None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

In [7]:
# bathrooms, sqft_living, sqft_living_sqrt

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [9]:
import numpy as np
l1_penaltys = np.logspace(1, 7, num=13)

In [10]:
for l1_penalty in l1_penaltys:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty,verbose = False)
    predict  = model.predict(validation)
    tmp = predict - validation['price']
    rss = np.dot(tmp, tmp).sum()
    print 'current l1_penalty:'+str(l1_penalty)+', current RSS:'+str(rss)

current l1_penalty:10.0, current RSS:6.25766285142e+14
current l1_penalty:31.6227766017, current RSS:6.25766285362e+14
current l1_penalty:100.0, current RSS:6.25766286058e+14
current l1_penalty:316.227766017, current RSS:6.25766288257e+14
current l1_penalty:1000.0, current RSS:6.25766295212e+14
current l1_penalty:3162.27766017, current RSS:6.25766317206e+14
current l1_penalty:10000.0, current RSS:6.25766386761e+14
current l1_penalty:31622.7766017, current RSS:6.25766606749e+14
current l1_penalty:100000.0, current RSS:6.25767302792e+14
current l1_penalty:316227.766017, current RSS:6.25769507644e+14
current l1_penalty:1000000.0, current RSS:6.25776517727e+14
current l1_penalty:3162277.66017, current RSS:6.25799062845e+14
current l1_penalty:10000000.0, current RSS:6.25883719085e+14


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [11]:
1000000

1000000

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [12]:
model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1000000,verbose = False)
coefficients = model.get("coefficients")
coefficients[coefficients['value']!=0]['value']

dtype: float
Rows: ?
[19054.555576410712, 7935.532947116086, 936.1052541559584, 25407.90440016528, 39.114453235557704, 1124.580426079912, 0.0028947664010812264, 148.21845770879176, 21199.337322637606, 12911.076230195722, 601568.8759734436, 93290.50227037004, 6606.5965213227455, 6206.283318341521, 43.284627929774054, 122.33376826175702, 9.429781422240257, 55.997517223545174, ... ]

In [13]:
coefficients['value'].nnz()

18

In [14]:
coefficients.num_rows

<bound method SFrame.num_rows of Columns:
	name	str
	index	str
	value	float
	stderr	float

Rows: 18

Data:
+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  19054.5555764   |  None  |
|     bedrooms     |  None |  7935.53294712   |  None  |
| bedrooms_square  |  None |  936.105254156   |  None  |
|    bathrooms     |  None |  25407.9044002   |  None  |
|   sqft_living    |  None |  39.1144532356   |  None  |
| sqft_living_sqrt |  None |  1124.58042608   |  None  |
|     sqft_lot     |  None | 0.00289476640108 |  None  |
|  sqft_lot_sqrt   |  None |  148.218457709   |  None  |
|      floors      |  None |  21199.3373226   |  None  |
|  floors_square   |  None |  12911.0762302   |  None  |
+------------------+-------+------------------+--------+
[18 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good 
value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [15]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [16]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [17]:
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty,verbose = False)
    predict  = model.predict(validation)
    tmp = predict - validation['price']
    rss = np.dot(tmp, tmp).sum()
    print 'current l1_penalty:'+str(l1_penalty)+', current nonzeros:'+str(model['coefficients']['value'].nnz())+', current RSS:'+str(rss)

current l1_penalty:100000000.0, current nonzeros:18, current RSS:6.27492659875e+14
current l1_penalty:127427498.57, current nonzeros:18, current RSS:6.28210516771e+14
current l1_penalty:162377673.919, current nonzeros:18, current RSS:6.29176689541e+14
current l1_penalty:206913808.111, current nonzeros:18, current RSS:6.30650082719e+14
current l1_penalty:263665089.873, current nonzeros:17, current RSS:6.32940229287e+14
current l1_penalty:335981828.628, current nonzeros:17, current RSS:6.3626814023e+14
current l1_penalty:428133239.872, current nonzeros:17, current RSS:6.41261198311e+14
current l1_penalty:545559478.117, current nonzeros:17, current RSS:6.48983455376e+14
current l1_penalty:695192796.178, current nonzeros:17, current RSS:6.60962217696e+14
current l1_penalty:885866790.41, current nonzeros:16, current RSS:6.77261520728e+14
current l1_penalty:1128837891.68, current nonzeros:15, current RSS:7.01046815867e+14
current l1_penalty:1438449888.29, current nonzeros:15, current RSS:7.3

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [18]:
l1_penalty_min = 2976351441.63
l1_penalty_max = 3792690190.73

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [19]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [20]:
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty,verbose = False)
    predict  = model.predict(validation)
    tmp = predict - validation['price']
    rss = np.dot(tmp, tmp).sum()
    print 'current l1_penalty:'+str(l1_penalty)+', current nonzeros:'+str(model['coefficients']['value'].nnz())+', current RSS:'+str(rss)

current l1_penalty:2976351441.63, current nonzeros:10, current RSS:9.66925692362e+14
current l1_penalty:3019316638.95, current nonzeros:10, current RSS:9.74019450085e+14
current l1_penalty:3062281836.27, current nonzeros:10, current RSS:9.81188367942e+14
current l1_penalty:3105247033.59, current nonzeros:10, current RSS:9.89328342459e+14
current l1_penalty:3148212230.91, current nonzeros:10, current RSS:9.98783211266e+14
current l1_penalty:3191177428.24, current nonzeros:10, current RSS:1.00847716702e+15
current l1_penalty:3234142625.56, current nonzeros:10, current RSS:1.01829878055e+15
current l1_penalty:3277107822.88, current nonzeros:10, current RSS:1.02824799221e+15
current l1_penalty:3320073020.2, current nonzeros:8, current RSS:1.03461690923e+15
current l1_penalty:3363038217.52, current nonzeros:8, current RSS:1.03855473594e+15
current l1_penalty:3406003414.84, current nonzeros:8, current RSS:1.04323723787e+15
current l1_penalty:3448968612.16, current nonzeros:7, current RSS:1.0

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [21]:
max_nonzeros_l1_penalty = 3620829401.45

In [22]:
model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=max_nonzeros_l1_penalty,verbose = False)

In [23]:
coefficients = model['coefficients']

In [24]:
coefficients[coefficients['value']!=0]['name']

dtype: str
Rows: ?
['(intercept)', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'grade', 'sqft_above', ... ]